In [2]:
import os
import json
import pandas as pd
import traceback

In [5]:
from langchain.chat_models import ChatOpenAI

In [6]:
from dotenv import load_dotenv # This helps to have access the file .env created to read all value variables we have created

load_dotenv()  # take environment variables

True

In [24]:
KEY=os.getenv("testing")

In [25]:
llm = ChatOpenAI(openai_api_key=KEY, model_name="gpt-3.5-turbo", temperature=0.5)

In [26]:
# Import all necessaries libraries
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [27]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    
}

In [28]:
TEMPLATE="""
Text: {text}
You are an expert MCQ maker. Given the above text, it in your job \
to create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your questions like RESPONSE_JSON below and use it as a guide.\
Ensure to make {number} MCQs

### RESPONSE_JSON
{response_json}
"""

In [29]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE

)

In [30]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [31]:
# Develop an evaluation template for my quizzes. 
TEMPLATE2="""
You are expert english grammarian and writer. Given a Mutiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity\
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the students abilitities.\
Quiz_MCQs:
{quiz}

check from an expert English writer of the above quiz:
"""

In [32]:
quiz_evaluation_prompt = PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE2)

In [33]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [34]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain,review_chain], input_variables=["text", "number", "subject", "tone", "response_json"], 
                output_variables=["quiz", "review"], verbose=True,)

In [35]:
file_path = r"C:\Users\Hp\Documents\github\mcqstack\data.txt"

In [36]:
file_path

'C:\\Users\\Hp\\Documents\\github\\mcqstack\\data.txt'

In [37]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [38]:
print(TEXT)

The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[12] Other researchers who have studied human cognitive systems contributed 

In [39]:
# Serialize the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [40]:
NUMBER=5
SUBJECT="machine learning"
Tone="simple"

In [41]:
# https://python.langchain.com/v0.1/docs/modules/model_io/llms/token_usage_tracking/
# How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject": SUBJECT,
            "tone": Tone,
            "response_json": json.dumps(RESPONSE_JSON)
        }
    )

C:\Users\Hp\AppData\Local\Temp\ipykernel_4836\447738503.py:4: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  response=generate_evaluate_chain(
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Prompt after formatting:

Text: The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[12] Other researchers who have studied huma

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



> Finished chain.
Prompt after formatting:

You are expert english grammarian and writer. Given a Mutiple Choice Quiz for machine learning students.You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexityif the quiz is not at per with the cognitive and analytical abilities of the students,update the quiz questions which needs to be changed and change the tone such that it perfectly fits the students abilitities.Quiz_MCQs:
### RESPONSE_JSON
{"1": {"mcq": "Who coined the term machine learning in 1959?", "options": {"a": "Donald Hebb", "b": "Tom M. Mitchell", "c": "Arthur Samuel", "d": "Walter Pitts"}, "correct": "c"}, "2": {"mcq": "What was the name of the experimental 'learning machine' developed by Raytheon Company in the early 1960s?", "options": {"a": "Cyberdyne", "b": "Cybertron", "c": "Skynet", "d": "Matrix"}, "correct": "b"}, "3": {"mcq": "Who proposed the early mathematical models of neural networks in 

In [42]:
print(f"Total tokens: {cb.total_tokens}")
print(f"Prompt tokens: {cb.prompt_tokens}")
print(f"Completion Tokens: {cb.completion_tokens}")
print(f"Total Cost: {cb.total_cost}")

Total tokens: 1750
Prompt tokens: 1355
Completion Tokens: 395
Total Cost: 0.0028225


In [43]:
response

{'text': 'The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]\n\nAlthough the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb\'s model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[12] Other researchers who have studied human cognitive systems

In [47]:
quiz=response.get("quiz")

In [54]:
quiz=quiz.split('\n', 1)[1]


In [55]:
quiz=json.loads(quiz)

In [56]:
quiz

{'1': {'mcq': 'Who coined the term machine learning in 1959?',
  'options': {'a': 'Donald Hebb',
   'b': 'Tom M. Mitchell',
   'c': 'Arthur Samuel',
   'd': 'Walter Pitts'},
  'correct': 'c'},
 '2': {'mcq': "What was the name of the experimental 'learning machine' developed by Raytheon Company in the early 1960s?",
  'options': {'a': 'Cyberdyne',
   'b': 'Cybertron',
   'c': 'Skynet',
   'd': 'Matrix'},
  'correct': 'b'},
 '3': {'mcq': 'Who proposed the early mathematical models of neural networks in the study of human cognitive systems?',
  'options': {'a': 'Warren McCulloch',
   'b': 'Ray Kurzweil',
   'c': 'Elon Musk',
   'd': 'Alan Turing'},
  'correct': 'a'},
 '4': {'mcq': 'According to Tom M. Mitchell, what does a computer program do in the context of machine learning?',
  'options': {'a': 'Think like a human',
   'b': 'Learn from experience',
   'c': 'Predict future outcomes',
   'd': 'Improve performance measure'},
  'correct': 'b'},
 '5': {'mcq': 'What are the two main objecti

In [57]:
# Create a dataframe
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [f"{option}: {option_value}" for option, option_value in value["options"].items()]
    )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [58]:
quiz=pd.DataFrame(quiz_table_data)

In [59]:
quiz

,MCQ,Choices,Correct
0,Who coined the term machine learning in 1959?,a: Donald Hebb | b: Tom M. Mitchell | c: Arthu...,c
1,What was the name of the experimental 'learnin...,a: Cyberdyne | b: Cybertron | c: Skynet | d: M...,b
2,Who proposed the early mathematical models of ...,a: Warren McCulloch | b: Ray Kurzweil | c: Elo...,a
3,"According to Tom M. Mitchell, what does a comp...",a: Think like a human | b: Learn from experien...,b
4,What are the two main objectives of modern-day...,a: To play games and chat online | b: To class...,b


In [60]:
quiz.to_csv("machinelearning.csv", index=False)